# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
import requests
from datetime import datetime


API_KEY = "ahTKQej4yzOhQeippM2UpYTxsOOe1CEfmdUqXW1y"
BASE_URL = "https://api.nasa.gov/"


def get_apod():
    url = f"{BASE_URL}planetary/apod?api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        print(f"Дата: {data['date']}")
        print(f"Тема: {data['title']}")
        print(f"Пояснение: {data['explanation']}")
        print(f"Ссылка на изображение: {data['hdurl']}")
    else:
        print("Ошибка при получении APOD.")


def get_mars_rover_photos(date,rover,camera):
    rovers = ["curiosity", "opportunity", "spirit"]
    cameras = {
        "FHAZ": "Фронтальная камера",
        "RHAZ": "Задняя камера",
        "MAST": "Главная камера",
        "CHEMCAM": "Камера химического анализа",
        "MAHLI": "Микроскопический инструмент",
        "MARDI": "Марсианская десантная камера",
        "NAVCAM": "Навигационная камера",
        "PANCAM": "Панорамная камера",
        "MINITES": "Миниатюрный спектрометр теплового излучения"
    }

    url = f"{BASE_URL}mars-photos/api/v1/rovers/{rover}/photos?earth_date={date}&api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        photos = data["photos"]

        if len(photos) > 0:
            print(f"\nФотографии с марсохода {rover.upper()} за {date}:")

            for photo in photos:
                if photo['camera']['name']==camera:
                    print(f"- Камера: {camera}, ID фото: {photo['id']}, Ссылка: {photo['img_src']}")
        else:
            print(f"Нет фотографий с марсохода {rover.upper()} за {date}.")
    else:
        print(f"Ошибка при получении фотографий с марсохода {rover.upper()}.")

def get_neo(date):
    url = f"{BASE_URL}neo/rest/v1/feed?start_date={date}&end_date={date}&api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        near_earth_objects = data["near_earth_objects"].get(date, [])

        if len(near_earth_objects) > 0:
            print(f"\nОбъекты, сближающиеся с Землёй на {date}:")

            for neo in near_earth_objects:
                name = neo["name"]
                estimated_diameter_min = neo["estimated_diameter"]["meters"]["estimated_diameter_min"]
                estimated_diameter_max = neo["estimated_diameter"]["meters"]["estimated_diameter_max"]
                is_potentially_hazardous_asteroid = neo["is_potentially_hazardous_asteroid"]

                print(f"- Название: {name}")
                print(f" Диаметр: от {estimated_diameter_min:.2f} до {estimated_diameter_max:.2f} метров")
                print(f" Потенциально опасный астероид: {'Да' if is_potentially_hazardous_asteroid else 'Нет'}")
        else:
            print(f"Нет объектов, сближающихся с Землёй на {date}.")
    else:
        print("Ошибка при получении информации об объектах, сближающихся с Землёй.")

def get_space_weather():
    api_url = "https://api.nasa.gov/DONKI/notifications"

    params = {
        "startDate": "2023-10-01",
        "endDate": "2023-11-30",
        "type": "FLARE",  # Тип события (солнечные вспышки)
        "api_key": API_KEY
    }

    params_2 = {
        "startDate": "2023-10-01",
        "endDate": "2023-11-30",
        "type": "GST",  # Тип события (солнечные вспышки)
        "api_key": API_KEY
    }

    response = requests.get(url=api_url, params=params)
    response_2=requests.get(url=api_url, params=params_2)

    if response.status_code == 200:
        data = response.json()

        print(data[0]['messageBody'])
    if response_2.status_code == 200:
        data_2 = response_2.json()

        print(data_2[0]['messageBody'])
    else:
        print("Ошибка при получении данных о космической погоде.")

def validate_date(input_date):
    try:
        date = datetime.strptime(input_date, "%Y-%m-%d").strftime("%Y-%m-%d")
        return date
    except ValueError:
        print("Неверный формат даты. Пожалуйста, используйте формат ГГГГ-ММ-ДД.")
        return None

def main_menu():
    while True:
        print("\nГлавное меню:")
        print("1. Астрономическая картинка дня (APOD)")
        print("2. Фотографии с марсохода")
        print("3. Объекты, сближающиеся с Землёй (ОСЗ)")
        print("4. Данные о космической погоде")
        print("5. Выход")

        choice = input("Выберите пункт меню: ")

        if choice == "1":
            get_apod()
        elif choice == "2":
            date = input("Введите дату в формате ГГГГ-ММ-ДД: ")
            validated_date = validate_date(date)
            if validated_date:
                get_mars_rover_photos(validated_date,input('Введите назване марсохода:'),input('Введите название камеры'))
        elif choice == "3":
            date = input("Введите дату в формате ГГГГ-ММ-ДД: ")
            validated_date = validate_date(date)
            if validated_date:
                get_neo(validated_date)
        elif choice == "4":
            get_space_weather()
        elif choice == "5":
            break
        else:
            print("Неправильный выбор. Попробуйте снова.")

if __name__ == "__main__":
    main_menu()


Главное меню:
1. Астрономическая картинка дня (APOD)
2. Фотографии с марсохода
3. Объекты, сближающиеся с Землёй (ОСЗ)
4. Данные о космической погоде
5. Выход
Выберите пункт меню: 1
Дата: 2024-11-08
Тема: Helping Hand in Cassiopeia
Пояснение: Drifting near the plane of our Milky Way galaxy these dusty molecular clouds seem to extend a helping hand on a cosmic scale. Part of a local complex of star-forming interstellar clouds they include LDN 1358, 1357, and 1355 from American astronomer Beverly Lynds' 1962 Catalog of Dark Nebulae. Presenting a challenging target for astro-imagers, the obscuring dark nebulae are nearly 3,000 light-years away, toward rich starfields in the northern constellation Cassiopeia. At that distance, this deep, telescopic field of view would span about 80 light-years.
Ссылка на изображение: https://apod.nasa.gov/apod/image/2411/Ldn1355HelpingHandFR2048.png

Главное меню:
1. Астрономическая картинка дня (APOD)
2. Фотографии с марсохода
3. Объекты, сближающиеся с 

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests
BASE_URL = "https://api.artic.edu/api/v1/artworks"


def fetch_artworks(page=1):
    params = {"page": page}
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        return data["data"], data["pagination"]
    else:
        print(f"Ошибка при получении данных: {response.status_code}")
        return [], {}


def filter_by_artist(artworks, artist_name):
    filtered_artworks = [artwork for artwork in artworks if artist_name in artwork["artist_titles"]]
    return filtered_artworks


def display_detail(artwork_id):
    url = f"{BASE_URL}/{artwork_id}"
    response = requests.get(url)
    if response.status_code == 200:
        detail_data = response.json()
        title = detail_data["data"]["title"]
        artist = detail_data["data"]["artist_title"]
        date_display = detail_data["data"]["date_display"]
        medium_display = detail_data["data"]["medium_display"]

        print("\nПодробная информация:")
        print(f"Название: {title}")
        print(f"Исполнитель: {artist}")
        print(f"Дата: {date_display}")
        print(f"Носитель: {medium_display}\n")
    else:
        print(f"Не удалось получить данные о произведении: {response.status_code}")

def display_artworks(artworks):
    for i, artwork in enumerate(artworks):
        print(f"{i + 1}. {artwork['title']}")

def main():
    current_page = 1
    while True:
        artworks, pagination = fetch_artworks(current_page)

        if not artworks:
            print("Нет произведений искусства.")
            break

        print(f"\nСтраница {current_page} из {pagination['total_pages']} | Всего произведений: {pagination['total']}")
        display_artworks(artworks)

        action = input("\nВведите номер произведения для просмотра деталей, 'next' для следующей страницы, 'prev' для предыдущей страницы, 'filter' для фильтрации по художнику или 'exit' для выхода: ").strip().lower()

        if action.isdigit():
            index = int(action) - 1
            if 0 <= index < len(artworks):
                artwork_id = artworks[index]["id"]
                display_detail(artwork_id)
            else:
                print("Неправильный индекс произведения.")
        elif action == "next":
            if current_page < pagination["total_pages"]:
                current_page += 1
            else:
                print("Вы достигли последней страницы.")
        elif action == "prev":
            if current_page > 1:
                current_page -= 1
            else:
                print("Вы находитесь на первой странице.")
        elif action == "filter":
            artist_name = input("Введите имя художника для фильтрации: ")
            filtered_artworks = filter_by_artist(artworks, artist_name)
            if filtered_artworks:
                display_artworks(filtered_artworks)
            else:
                print("По вашему запросу ничего не найдено.")
        elif action == "exit":
            print("Завершение программы...")
            break
        else:
            print("Некорректное действие. Попробуйте еще раз.")

if __name__ == "__main__":
    main()


Страница 1 из 10527 | Всего произведений: 126319
1. The Crucifixion
2. Two Putti
3. Fragment from Christ Carrying the Cross: Saint John the Evangelist
4. Elements I
5. Linear
6. 3-D Squares
7. The Tables Have Turned
8. Panel (Upholstery Fabric)
9. Kyoto Evergreen (Furnishing Fabric)
10. Girafters (Furnishing Fabric)
11. Bedcover
12. Rumble, from Brawling Spitfire Wrestling Series

Страница 2 из 10527 | Всего произведений: 126319
1. Bedcover
2. Rumble, from Brawling Spitfire Wrestling Series
3. Untitled, plate ten from See America First
4. Ceremonial Hanging (palepai)
5. Jelayah (Overcoat Dress)
6. Dress
7. Rose Bowl
8. Kimkhwab Textile
9. Kimkhwab Textile
10. Kimkhwab Textile
11. Kimkhwab Textile
12. Kimkhwab Textile

Подробная информация:
Название: Dress
Исполнитель: None
Дата: 1875/1925
Носитель: Cotton or linen; silk embroidery; mother of pearl button


Страница 2 из 10527 | Всего произведений: 126319
1. Bedcover
2. Rumble, from Brawling Spitfire Wrestling Series
3. Untitled, plate

# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

Получение различной информации о датах и числах

In [ ]:
import requests
import argparse
from datetime import datetime


def get_fact_by_number(number):
    url = f"http://numbersapi.com/{number}/math"
    response = requests.get(url).text

    print(f"Интересный факт о числе {number}:\n{response}")


def get_random_fact():
    url = "http://numbersapi.com/random/math"
    response = requests.get(url).text

    print(f"Случайный факт о математике:\n{response}")


def get_date_fact(date_str):
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        print("Некорректный формат даты. Используйте формат ГГГГ-ММ-ДД.")
        return

    year = date_obj.year
    month = date_obj.month
    day = date_obj.day

    url = f"http://numbersapi.com/{month}/{day}/date"
    response = requests.get(url).text

    print(f"Интересный факт о дате {date_str}:\n{response}")


def get_holidays(date_str):
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        print("Некорректный формат даты. Используйте формат ГГГГ-ММ-ДД.")
        return

    year = date_obj.year
    month = date_obj.month
    day = date_obj.day

    url = f"https://date.nager.at/api/v2/publicholidays/{year}/{month}-{day}"
    response = requests.get(url).json()

    holidays = [holiday['localName'] for holiday in response]

    if holidays:
        print(f"Праздники на дату {date_str}:")
        for holiday in holidays:
            print(holiday)
    else:
        print(f"На дату {date_str} нет известных праздников.")


def convert_to_roman(numeral):
    if not isinstance(numeral, int) or numeral < 1 or numeral > 3999:
        print("Число должно быть целым числом в диапазоне от 1 до 3999.")
        return

    roman_numerals = [
        ("M", 1000),
        ("CM", 900),
        ("D", 500),
        ("CD", 400),
        ("C", 100),
        ("XC", 90),
        ("L", 50),
        ("XL", 40),
        ("X", 10),
        ("IX", 9),
        ("V", 5),
        ("IV", 4),
        ("I", 1)
    ]

    result = ""
    for symbol, value in roman_numerals:
        while numeral >= value:
            result += symbol
            numeral -= value

    print(f"Римское представление числа {numeral}: {result}")


def main():
    while True:
        print('Выберите номер действия:')
        print('1. Интересного факта о конкретном числе')
        print('2. Случайный факт о любом числе')
        print('3. Факт о конкретной дате')
        print('4. Список праздников на конкретную дату')
        print('5. Перевода числа в римскую систему счисления')
        print('6. Выход')

        try:
            action = int(input())

            if action == 1:
                number = int(input("Введите число: "))
                get_fact_by_number(number)
            elif action == 2:
                get_random_fact()
            elif action == 3:
                date_str = input("Введите дату (ГГГГ-ММ-ДД): ")
                get_date_fact(date_str)
            elif action == 4:
                date_str = input("Введите дату (ДД.ММ.ГГГГ): ")
                get_holidays(date_str)
            elif action == 5:
                numeral = int(input("Введите число: "))
                convert_to_roman(numeral)
            elif action == 6:
                print('Завершение программы')
                break

        except ValueError:
            print('Некорректный ввод, попробуйте ещё раз')


if __name__ == "__main__":
    main()

Выберите номер действия:
1. Интересного факта о конкретном числе
2. Случайный факт о любом числе
3. Факт о конкретной дате
4. Список праздников на конкретную дату
5. Перевода числа в римскую систему счисления
6. Выход
1
Введите число: 5
Интересный факт о числе 5:
5 is the number of Platonic solids.
Выберите номер действия:
1. Интересного факта о конкретном числе
2. Случайный факт о любом числе
3. Факт о конкретной дате
4. Список праздников на конкретную дату
5. Перевода числа в римскую систему счисления
6. Выход
2
Случайный факт о математике:
2090 is the number of possible rows in a 17×17 crossword puzzle.
Выберите номер действия:
1. Интересного факта о конкретном числе
2. Случайный факт о любом числе
3. Факт о конкретной дате
4. Список праздников на конкретную дату
5. Перевода числа в римскую систему счисления
6. Выход
3
Введите дату (ДД.ММ.ГГГГ): 24.02.2006
Некорректный формат даты. Используйте формат ГГГГ-ММ-ДД.
Выберите номер действия:
1. Интересного факта о конкретном числе
2. Слу